In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import math
import numpy as np
import arrow
import cmocean

In [2]:
%matplotlib inline

In [3]:
ds = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')
ds

<xarray.Dataset>
Dimensions:    (gridX: 256, gridY: 266, time: 36456)
Coordinates:
  * time       (time) datetime64[ns] 2014-09-12 2014-09-12T01:00:00 ...
  * gridY      (gridY) float64 0.0 2.5e+03 5e+03 7.5e+03 1e+04 1.25e+04 ...
  * gridX      (gridX) float64 0.0 2.5e+03 5e+03 7.5e+03 1e+04 1.25e+04 ...
Data variables:
    atmpres    (time, gridY, gridX) float32 ...
    precip     (time, gridY, gridX) float32 ...
    qair       (time, gridY, gridX) float32 ...
    solar      (time, gridY, gridX) float32 ...
    tair       (time, gridY, gridX) float32 ...
    therm_rad  (time, gridY, gridX) float32 ...
    u_wind     (time, gridY, gridX) float32 ...
    v_wind     (time, gridY, gridX) float32 ...
Attributes:
    acknowledgement:           MEOPAR, ONC, Compute Canada, Environment Canada
    cdm_data_type:             Grid
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    coverage_content_type:     modelResult
    creator_email:             sallen@eos.ubc.ca
    creator_name: 

In [4]:
ds.time[0]

<xarray.DataArray 'time' ()>
array(1410480000000000000, dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2014-09-12
Attributes:
    _ChunkSizes:                 1
    _CoordinateAxisType:         Time
    actual_range:                [  1.41048000e+09   1.54171800e+09]
    axis:                        T
    ioos_category:               Time
    long_name:                   verification time generated by wgrib2 functi...
    reference_date:              2018.11.07 18:00:00 UTC
    reference_time:              1541613600.0
    reference_time_description:  kind of product unclear, reference date is v...
    reference_time_type:         0
    standard_name:               time
    time_origin:                 01-JAN-1970 00:00:00
    time_step:                   3600.0
    time_step_setting:           auto

In [5]:
ds.time[-1]

<xarray.DataArray 'time' ()>
array(1541718000000000000, dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2018-11-08T23:00:00
Attributes:
    _ChunkSizes:                 1
    _CoordinateAxisType:         Time
    actual_range:                [  1.41048000e+09   1.54171800e+09]
    axis:                        T
    ioos_category:               Time
    long_name:                   verification time generated by wgrib2 functi...
    reference_date:              2018.11.07 18:00:00 UTC
    reference_time:              1541613600.0
    reference_time_description:  kind of product unclear, reference date is v...
    reference_time_type:         0
    standard_name:               time
    time_origin:                 01-JAN-1970 00:00:00
    time_step:                   3600.0
    time_step_setting:           auto

## Slice JFM time and then converts xarray into numpy array using ".values"

In [6]:
jan2015_time = ds.time.sel(time=slice("2015-01-01","2015-01-31")).values

## Calculate percent time that january 2015 wind speeds are > 10 m/s

In [7]:
jan2015_wspd = np.sqrt(ds.u_wind.sel(time = jan2015_time)**2 + ds.v_wind.sel(time = jan2015_time)**2)

thresh = 7
wspd_size = jan2015_wspd.shape
total = wspd_size[0]

# Create numpy array for percentage value output
wspd_2D_percent = np.zeros((wspd_size[1],wspd_size[2]))
for iy in range(0,wspd_size[1]):
    for ix in range(0,wspd_size[2]):
        wspd_ts = jan2015_wspd[:,iy,ix]
        wspd_ts_thresh =  wspd_ts[wspd_ts > thresh]
        wspd_2D_percent[iy,ix] = 100 * len(wspd_ts_thresh) / total


IndexError: The indexing operation you are attempting to perform is not valid on netCDF4.Variable object. Try loading your data into memory first by calling .load().

In [ ]:
total

In [ ]:
wspd_size

## plot up results

In [ ]:
fig, ax = plt.subplots()
cax = ax.imshow(np.flipud(wspd_2D_percent))
ax.set_title('Percent time with wind speed over 10 m/s for Jan. 2015')
cbar = fig.colorbar(cax, orientation='vertical')
cbar.ax.set_ylabel('%')

## now try to figure out how to index months by converting numeric month into text index

In [ ]:
month = 1
time_01_15 = ds.time.sel(time=slice("2015-01-01","2015-01-31")).values
strmn = str(month)
strmn

In [ ]:
from datetime import date
test = date(2015,1,1)
test

In [ ]:
test.strftime("%y-%m-%d")

In [ ]:
test.strftime("%Y-%m-%d")

In [ ]:
# old way: specify date
# jan2015_time = ds.time.sel(time=slice("2015-01-01","2015-01-31")).values

# new way: Use numeric date value (in preparation to create a loop)
t1 = date(2015,1,1)
t2 = date(2015,1,1)
time_period = ds.time.sel(time=slice(t1.strftime("%Y-%m-%d"),t2.strftime("%Y-%m-%d"))).values
time_period

In [ ]:
jan2015_wspd = np.sqrt(ds.u_wind.sel(time = time_period)**2 + ds.v_wind.sel(time = time_period)**2)

thresh = 7
wspd_size = jan2015_wspd.shape
total = wspd_size[0]

In [ ]:
# Create numpy array for percentage value output
wspd_2D_percent = np.zeros((wspd_size[1],wspd_size[2]))
for iy in range(0,wspd_size[1]):
    for ix in range(0,wspd_size[2]):
        wspd_ts = jan2015_wspd[:,iy,ix]
        wspd_ts_thresh =  wspd_ts[wspd_ts > thresh]
        wspd_2D_percent[iy,ix] = 100 * len(wspd_ts_thresh) / total

In [ ]:
fig, ax = plt.subplots()
cax = ax.imshow(np.flipud(wspd_2D_percent))
ax.set_title('Percent time with wind speed over 10 m/s for Jan. 2015')
cbar = fig.colorbar(cax, orientation='vertical')
cbar.ax.set_ylabel('%')